# Attacking and Defending Active Directory

To download and load the AD Powershell Module
docs.microsoft.com/en-us/powershell/module/addsadministration/?view=win10-ps

Import-Module .\<filename>.dll

-- Question is can the module be loaded

## Global Settings

In [26]:
destIP = "192.168.113.161"
parentDir = "/home/kali/hackthebox/output"
box = "Win10"
boxDir = parentDir + "/" + box
!mkdir $boxDir
winrmusername = "thepcn3rd"
winrmpassword = ""

# Journal
journalPath = boxDir + "/journal.txt"


def posh(cmd):
    scriptPath = boxDir + "/ps.ps1"
    f = open(scriptPath, "w")
    fj = open(journalPath, "a")
    fj.write("\n\n####### Command: " + cmd + "\n")
    fj.write("################################################   Output  #####################################\n")
    f.write("#!/usr/bin/pwsh\n")
    f.write("$pw = Convertto-Securestring -AsPlainText -Force -String \"" + winrmpassword + "\"\n")
    f.write("$cred = new-object -typename System.Management.Automation.PSCredential -argumentlist \"" + winrmusername + "\",$pw" + "\n")
    f.write("$s = New-PSSession -Computername " + box + " -Authentication Negotiate -Credential $cred\n")
    # Enter pssession $s is a manual way
    f.write("Invoke-Command -Session $s {" + cmd + "}\n")
    #f.write("Invoke-Command -Session $s {" + poshCommand2 + "}\n")
    #f.write("Invoke-Command -Session $s {" + poshCommand3 + "}\n")
    f.close()
    !chmod 700 $scriptPath
    output = !$scriptPath
    for line in output:
        print(line)
        fj.write(str(line) + "\n")
    fj.close()
    #print(output)
    print("Completed Commands!")





mkdir: cannot create directory ‘/home/kali/hackthebox/output/Win10’: File exists


## Index
<a id="index"></a>

[User Enumeration](#userenum)

[Domain Enumeration](#domainenum)

<a id="userenum"></a>
## User Enumeration

[Back to Index](#index)

In [27]:
poshCommand = "whoami; whoami /groups; whoami /priv"
posh(poshCommand)

win10\thepcn3rd

GROUP INFORMATION
-----------------

Group Name                                                    Type             SID          Attributes                                                     
============================================================= ================ ============ ===============================================================
Everyone                                                      Well-known group S-1-1-0      Mandatory group, Enabled by default, Enabled group             
NT AUTHORITY\Local account and member of Administrators group Well-known group S-1-5-114    Mandatory group, Enabled by default, Enabled group             
BUILTIN\Administrators                                        Alias            S-1-5-32-544 Mandatory group, Enabled by default, Enabled group, Group owner
BUILTIN\Users                                                 Alias            S-1-5-32-545 Mandatory group, Enabled by default, Enabled group             
NT AUTHORI

<a id="domainenum"></a>
## Domain Enumeration

[Back to Index](#index)

In [29]:
# Gather the current domain name of the computer
# Also shows the Child Domains and Parent if exists
poshCommand = "$ADClass = [System.DirectoryServices.ActiveDirectory.Domain]"
poshCommand += "; $ADClass::GetCurrentDomain()"
posh(poshCommand)

MethodInvocationException: Exception calling "GetCurrentDomain" with "0" argument(s): "Current security context is not associated with an Active Directory domain or forest."
Completed Commands!


## With AD Module Loaded
Could load Powerview

#Get Domain Controllers from another domain

Get-ADDomainController -Domain --domain--

Get-DomainPolicy

(Get-DomainPolicy)."--name of policy--"

(Get-DomainPolicy -domain --domain--)."--name of policy--"

#Display all users in the current domain

Get-ADUser -Filter * - Properties *

Get-ADUser -Filter * -Properties * | SELECT name, @{expression {[datetime]::fromFileTime($_.pwdlastset)}}

# Evaluate badpwdcount in the properties for accounts outside the norm, these could be honeypots...

# Evaluate the number of "logoncount"s of the users
